In [103]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np

In [106]:
# loading businesses and restaurant ids
df_res = pd.read_csv("/Users/dima/code/Dimasaur/scorecast/data/restaurants_limited_features.csv")
res_ids = pd.read_csv("/Users/dima/code/Dimasaur/scorecast/data/restaurants.csv")

/var/folders/fd/585jf60x39vflmpy4gfz7yzh0000gn/T/ipykernel_43335/237053628.py:2: DtypeWarning: Columns (14,16) have mixed types. Specify dtype option on import or set low_memory=False.
  df_res = pd.read_csv("/Users/dima/code/Dimasaur/scorecast/data/restaurants_limited_features.csv")


In [107]:
df_res['food_type_string'] = df_res['food_type'].apply(lambda x: str(x).strip('[]'))
df_res['food_type_string'] = df_res['food_type_string'].apply(lambda x: str(x).replace("'",""))

In [111]:
df_bus = pd.read_csv("/Users/dima/code/Dimasaur/scorecast/raw_data/yelp_business_raw.csv")

In [113]:
res_ids.shape

(52268, 2)

In [119]:
df_bus.shape

(150346, 14)

In [55]:
df_res.food_type_string.value_counts(normalize=True).head(20)

food_type_string
O                         0.624570
Pizza                     0.033768
Mexican                   0.027416
Chinese                   0.026020
Burgers, Fast Food        0.015937
Italian, Pizza            0.012972
Italian                   0.011843
American (New)            0.010293
American (Traditional)    0.009700
Fast Food, Sandwiches     0.009700
Japanese, Sushi Bars      0.006275
                          0.006046
Thai                      0.005606
Sandwiches                0.005491
Barbeque                  0.005453
Vietnamese                0.005204
Seafood                   0.005108
Diners                    0.005089
Coffee & Tea, Food        0.004458
Indian                    0.004420
Name: proportion, dtype: float64

In [56]:
replace_dictionary = {
    "Pizza" : "Italian",
    "Burgers, Fast Food" : "Burgers",
    "Italian, Pizza" : "Italian",
    "American (New)" : "American",
    "Fast Food, Sandwiches" : "Sandwiches",
    "American (Traditional)" : "American",
    "Fast Food, Mexican" : "Mexican",
    "Coffee & Tea, Food" : "Cafes",
    "Japanese, Sushi Bars" : "Japanese",
    "American (Traditional), Diners" : "Diners",
    "Bakeries, Food" : "Bakery",
    "Delis, Sandwiches" : "Delis",
    "Mexican, Tex-Mex" : "Mexican",
    "American (Traditional), Burgers" : "American",
    "Sushi Bars" : "Japanese",
    "Chinese, Fast Food" : "Chinese",
    "American (Traditional), Cafes" : "American",
    "Mexican, Tacos" : "Mexican",
    "Pizza, Sandwiches" : "Sandwiches",
    "Food, Pizza" : "Italian",
    "American (Traditional), Seafood" : "American",
    "Greek, Mediterranean" : "Greek",
    "American (New), American (Traditional)" : "American",
    "Cajun/Creole, Seafood" : "Cajun/Creole"

}

In [115]:
df_res.food_type_string = df_res.food_type_string.replace(replace_dictionary)

In [232]:
df_food_t20 = list(df_res.food_type_string.value_counts(normalize=True).head(25).index)
df_food_t20.remove('O')
df_food_t20.remove('')
df_food_t20 = df_food_t20[:20]
np_food_20 = np.array([df_food_t20])

ohe = OneHotEncoder(sparse_output=False)

In [154]:
df_restaurants = df_bus[df_bus.business_id.isin(res_ids.business_id)].reset_index(drop=True)

In [155]:
print(df_restaurants.shape,df_bus.shape, res_ids.shape)

(52268, 14) (150346, 14) (52268, 2)


In [205]:
df_restaurants['food_type_one'] = None
df_restaurants.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,food_type_one,categories_clean
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",None,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '...",None,"Burgers, Fast Food, Sandwiches, Food, Ice Crea..."
2,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': ""u'full_bar'"", '...","Pubs, Restaurants, Italian, Bars, American (Tr...",NaN,None,"Pubs, Restaurants, Italian, Bars, American (Tr..."
3,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ""'casual'"", 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '...",None,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,..."
4,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"{'Alcohol': ""'none'"", 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'...",None,"Vietnamese, Food, Restaurants, Food Trucks"


In [206]:
df_restaurants.categories_clean = df_restaurants.categories.str.replace(", ",",")

In [207]:
df_restaurants.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,food_type_one,categories_clean
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",None,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '...",None,"Burgers, Fast Food, Sandwiches, Food, Ice Crea..."
2,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': ""u'full_bar'"", '...","Pubs, Restaurants, Italian, Bars, American (Tr...",NaN,None,"Pubs, Restaurants, Italian, Bars, American (Tr..."
3,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ""'casual'"", 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '...",None,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,..."
4,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"{'Alcohol': ""'none'"", 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'...",None,"Vietnamese, Food, Restaurants, Food Trucks"


In [216]:
for word in df_restaurants.categories_clean[5].split():
    print(word)

American
(Traditional),Restaurants,Diners,Breakfast
&
Brunch


In [226]:
# function that extracts a category from a column in a dataframe if that category is present in a given list
def extract_category(categories, food_types):
    for food in categories.split():
        if food in food_types:
            return food
    return "Other"

df_restaurants['food_type_one'] = df_restaurants['categories_clean'].apply(lambda x: extract_category(x, df_food_t20))

In [227]:
df_restaurants.food_type_one.value_counts(normalize=True).head(20)

food_type_one
Other           0.564686
American        0.239152
Sandwiches      0.028354
Mexican         0.027876
Italian         0.021466
Chinese         0.021466
Burgers         0.018156
Seafood         0.015038
Delis           0.009815
Japanese        0.009509
Cafes           0.008342
Barbeque        0.006371
Thai            0.005663
Vietnamese      0.005319
Indian          0.004898
Diners          0.004209
Steakhouses     0.003788
Cajun/Creole    0.003061
Greek           0.002832
Name: proportion, dtype: float64

In [229]:
df_restaurants.food_type_one.value_counts(normalize=True).head(22)

food_type_one
Other           0.564686
American        0.239152
Sandwiches      0.028354
Mexican         0.027876
Italian         0.021466
Chinese         0.021466
Burgers         0.018156
Seafood         0.015038
Delis           0.009815
Japanese        0.009509
Cafes           0.008342
Barbeque        0.006371
Thai            0.005663
Vietnamese      0.005319
Indian          0.004898
Diners          0.004209
Steakhouses     0.003788
Cajun/Creole    0.003061
Greek           0.002832
Name: proportion, dtype: float64

In [231]:
df_restaurants.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,food_type_one,categories_clean
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...",Other,"Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,1,"{'BusinessParking': 'None', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-22:0', '...",Other,"Burgers, Fast Food, Sandwiches, Food, Ice Crea..."
2,k0hlBqXX-Bt0vf1op7Jr1w,Tsevi's Pub And Grill,8025 Mackenzie Rd,Affton,MO,63123,38.565165,-90.321087,3.0,19,0,"{'Caters': 'True', 'Alcohol': ""u'full_bar'"", '...","Pubs, Restaurants, Italian, Bars, American (Tr...",NaN,American,"Pubs, Restaurants, Italian, Bars, American (Tr..."
3,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,1,"{'RestaurantsAttire': ""'casual'"", 'Restaurants...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,...","{'Monday': '0:0-0:0', 'Tuesday': '6:0-21:0', '...",Other,"Ice Cream & Frozen Yogurt, Fast Food, Burgers,..."
4,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,NaN,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,1,"{'Alcohol': ""'none'"", 'OutdoorSeating': 'None'...","Vietnamese, Food, Restaurants, Food Trucks","{'Monday': '11:0-14:0', 'Tuesday': '11:0-14:0'...",Other,"Vietnamese, Food, Restaurants, Food Trucks"


In [236]:
ohe.fit(df_restaurants[['food_type_one']])

OneHotEncoder(sparse_output=False)

In [237]:
df_restaurants[ohe.get_feature_names_out()] = ohe.transform(df_restaurants[['food_type_one']])